# Devices cleaning

This task will fetch the devices raw data, perform data cleaning, store metadata information in mongo and store processed data in parquet files.

## Import libraries

In [1]:
import sys

sys.path.append('..')

In [2]:
import os
import yaml
import requests
import numpy as np
import pandas as pd

from pymongo import MongoClient

from utils import get_files, memory_usage, \
    correct_encoding, update_page_metadata

## Load the data

In [3]:
BASE_PATH = '../../'

CONFIG_DIR = os.path.join(BASE_PATH, 'config')
STORAGE_DIR = os.path.join(BASE_PATH, 'storage')
DATA_DIR = os.path.join(BASE_PATH, 'data')

config = yaml.load(open(os.path.join(CONFIG_DIR, 'env.yml')),
                   Loader=yaml.FullLoader)

In [4]:
model = 'devices'
model_storage = os.path.join(STORAGE_DIR, model)

if not os.path.exists(model_storage):
    os.makedirs(model_storage)

client = MongoClient('mongodb://{}:{}@{}:{}'.format(config['MONGO_USERNAME'], config['MONGO_PASSWORD'],
                                                    config['MONGO_HOST'], config['MONGO_PORT']))
metadata_db = client[config['MONGO_DATABASE']]

model_metadata = metadata_db[model]

model_metadata.delete_many({})
model_metadata.insert_one({'pages': []})

In [5]:
csv_files = get_files(os.path.join(DATA_DIR, 'raw', model, '*.csv'))

In [6]:
if not csv_files:
    raise FileNotFoundError(
        'Couldn\'t find any csv files! Please make sure the filepath exists')

In [7]:
df = pd.read_csv(csv_files[0], sep=';', parse_dates=['created_at'], nrows=100)

# sample rows to list columns and dtypes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   id            100 non-null    int64         
 1   model         100 non-null    object        
 2   manufacturer  100 non-null    object        
 3   brand         100 non-null    object        
 4   os_version    100 non-null    object        
 5   is_root       100 non-null    int64         
 6   created_at    100 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(4)
memory usage: 5.6+ KB


In [8]:
df.describe(include=[np.number])

,id,is_root
count,100.000000,100.000000
mean,50.500000,0.060000
std,29.011492,0.238683
min,1.000000,0.000000
25%,25.750000,0.000000
50%,50.500000,0.000000
75%,75.250000,0.000000
max,100.000000,1.000000


In [9]:
df.describe(exclude=[np.number], datetime_is_numeric=True)

,model,manufacturer,brand,os_version,created_at
count,100,100,100,100,100
unique,72,22,27,11,NaN
top,ALE-L21,samsung,samsung,7.0,NaN
freq,8,28,28,34,NaN
mean,NaN,NaN,NaN,NaN,2017-10-09 10:20:04.120000256
min,NaN,NaN,NaN,NaN,2017-10-09 03:42:19
25%,NaN,NaN,NaN,NaN,2017-10-09 10:17:53.500000
50%,NaN,NaN,NaN,NaN,2017-10-09 10:35:56
75%,NaN,NaN,NaN,NaN,2017-10-09 10:56:22.750000128
max,NaN,NaN,NaN,NaN,2017-10-09 11:14:12


In [10]:
df.sample(10, random_state=0)

,id,model,manufacturer,brand,os_version,is_root,created_at
26,27,ASUS_Z017D,asus,asus,7.0,0,2017-10-09 10:19:54
86,87,HUAWEI MT7-L09,HUAWEI,Huawei,6.0,0,2017-10-09 11:03:44
2,3,ASUS_X014D,asus,asus,5.1.1,0,2017-10-09 04:48:35
55,56,X5,DOOGEE,DOOGEE,5.1,0,2017-10-09 10:38:40
75,76,ALE-L21,HUAWEI,Huawei,6.0,0,2017-10-09 10:56:37
93,94,SM-A310F,samsung,samsung,7.0,0,2017-10-09 11:10:09
16,17,ALE-L21,HUAWEI,Huawei,6.0,0,2017-10-09 09:57:22
73,74,ONE A2003,OnePlus,OnePlus,6.0.1,0,2017-10-09 10:56:17
54,55,ONE A2003,OnePlus,OnePlus,6.0.1,0,2017-10-09 10:38:18
95,96,SM-N950F,samsung,samsung,7.1.1,0,2017-10-09 11:12:06


## Basic data pre-processing

In [11]:
def save_df_page(page, chunks, collection, exclude=['id']):
    # concatenate data chunks -> careful benchmark this
    df = pd.concat(chunks, axis=0, ignore_index=True)

    # store page metadata in mongo
    update_page_metadata(collection, df)

    # save output to a parquet file with brotli compression
    df.to_parquet(os.path.join(model_storage, '{}.{}.parquet'.format(model, page)), compression='brotli')

    return page + 1

In [12]:
mappings = {'id': 'uint32', 'model': 'category', 'manufacturer': 'category',
            'brand': 'category', 'os_version': 'category', 'is_root': 'bool'}

model_metadata.update_one({}, {'$set': {'mappings': mappings}})

In [13]:
total_rows = 0
total_memory_usage = 0.0
memory_usage_split = 0.0
last_id = 0
frequency_threshold = 0.001

page = 0
chunks = []
models_set = set()

In [14]:
print('Total csv files: {}'.format(len(csv_files)))

Total csv files: 2


In [15]:
for filepath in csv_files:
    df = pd.read_csv(filepath, sep=';', parse_dates=['created_at'])
    
    # brand
    df = df.drop(df[df['brand'].str.isnumeric()].index, axis=0)
    
    # manufacturer
    df = df.drop(df[df['manufacturer'].str.isnumeric()].index, axis=0)

    # os_version
    df = df.drop(df[df['os_version'].str.isalpha()].index, axis=0)
    
    # slice the minor os_version number e.g. 8.0.1 -> 8.0
    df.loc[:, 'os_version'] = ['.'.join(x.split('.')[:2]) for x in df['os_version']]
    
    # drop low frequency values
    value_counts = df['os_version'].value_counts(normalize=True)
    df = df[df['os_version'].isin(value_counts[value_counts >= frequency_threshold].index)]
    
    df.loc[:, 'os_version'] = df['os_version'].map(float).astype(str)
    
    # drop missing values
    df = df.dropna()
    
    string_columns = df.select_dtypes(include='object').columns.to_list()

    # basic string pre-processing: convert to lowercase and strip blank chars
    for column in string_columns:
        df.loc[:, column] = [x.lower().strip() for x in df[column]]

    df = df.astype(mappings)

    total_rows += df.shape[0]
    page_memory_usage = memory_usage(df)
    total_memory_usage += page_memory_usage
    memory_usage_split += page_memory_usage

    last_id = max(last_id, df['id'].max())
    models_set.update(df['model'].unique())

    if memory_usage_split >= config['MEMORY_USAGE_SPLIT']:
        print('Page {} created!'.format(page))
        page = save_df_page(page, chunks, model_metadata)

        del chunks[:]
        del chunks
        chunks = []
        memory_usage_split = 0.0
    else:
        chunks.append(df)
        
    print('{} ✔'.format(filepath))

../../data/raw/devices/devices.query.1.csv ✔
../../data/raw/devices/devices.query.2.csv ✔


In [16]:
if chunks:
    save_df_page(page, chunks, model_metadata)
    del chunks[:]
    del chunks

../utils.py:53: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  object_stats = df.describe(exclude=[np.number]).to_dict()


In [17]:
model_metadata.update_one({}, {'$set': {'last_id': int(last_id)}})
model_metadata.update_one({}, {'$set': {'total': total_rows}})

In [18]:
total_rows, total_memory_usage, last_id

(307141, 11.33, 307617)